In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('TPPHRM data and data processing//selected_factor_and_return.csv', index_col='date').drop(columns=['Unnamed: 0'])
df=df.dropna(subset=['Size', 'Liquidity', 'cpv_1mo', 'mr_1yr', 'mr_1w'], how='all')
df.fillna(0, inplace=True)

ret_1d = df[['Ticker','daily_return']]
data_1d = df.drop(columns=['daily_return'])

In [3]:
data_1d

,Ticker,Size,Liquidity,cpv_1mo,mr_1yr,mr_1w
date,,,,,,
2005-03-02,4151.T,0.603338,-0.370407,0.000000,0.000000,-1.381918
2005-03-03,4151.T,0.603338,-0.370407,0.426358,0.000000,-0.462423
2005-03-04,4151.T,0.603338,-0.370407,-1.005234,0.000000,-1.477961
2005-03-07,4151.T,0.603338,-0.370407,-0.731422,0.000000,-0.398283
2005-03-08,4151.T,0.603338,-0.370407,-1.081762,0.000000,-1.433256
...,...,...,...,...,...,...
2024-02-22,4887.T,0.052614,0.312916,-0.967226,-1.227663,1.173214
2024-02-26,4887.T,0.052614,0.312916,-1.061441,-1.209116,-0.017219
2024-02-27,4887.T,0.052614,0.312916,-1.163607,-1.202994,-0.882076


In [4]:
train_end_1 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.4)]
val_end_1 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.45)]
test_end_1 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.5)]
print(train_end_1,val_end_1,test_end_1)

2012-09-20 2013-09-03 2014-08-20


In [5]:
train_start_2 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.25)]
train_end_2 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.65)]
val_end_2 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.7)]
test_end_2 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.75)]
print(train_start_2,train_end_2,val_end_2,test_end_2)

2009-11-05 2017-07-06 2018-05-31 2019-05-13


In [6]:
train_start_3 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.5)]
train_end_3 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.9)]
val_end_3 = data_1d.index.unique()[int(len(data_1d.index.unique())*0.95)]
print(train_start_3,train_end_3,val_end_3)

2014-08-20 2022-03-31 2023-03-16


In [7]:
train_data_1 = data_1d[data_1d.index <= train_end_1]
val_data_1 = data_1d[(data_1d.index > train_end_1) & (data_1d.index <= val_end_1)]
test_data_1 = data_1d[(data_1d.index > val_end_1) & (data_1d.index <= test_end_1)]

train_data_2 = data_1d[(data_1d.index > train_start_2) & (data_1d.index <= train_end_2) ]
val_data_2 = data_1d[(data_1d.index > train_end_2) & (data_1d.index <= val_end_2)]
test_data_2 = data_1d[(data_1d.index > val_end_2) & (data_1d.index <= test_end_2)]

train_data_3 = data_1d[(data_1d.index > train_start_3) & (data_1d.index <= train_end_3) ]
val_data_3 = data_1d[(data_1d.index > train_end_3) & (data_1d.index <= val_end_3)]
test_data_3 = data_1d[(data_1d.index > val_end_3) ]


In [8]:
from tqdm import tqdm
def make_samples(data_1d, ret, seq_len):
    COLUMNS = data_1d.columns.drop('Ticker')
    sample_idxs = []
    df = data_1d.copy()
    for code, dt in tqdm(df.groupby('Ticker'), desc='rolling idx'):
         dt['date_'] = dt.index
         for ts in list(dt['date_'].rolling(seq_len))[seq_len-1:]:
             sample_idxs.append((code, ts.values.tolist()))
    x_1d, y, idx, codes =  [], [], [], []
    for code, times in tqdm(sample_idxs, desc='make_sample'):
        x_code_1d = data_1d[data_1d['Ticker']==code]
        ret_code = ret[ret['Ticker'] == code]
        x_1d.append(x_code_1d.loc[times, COLUMNS].values)
        y.append(ret_code.loc[times[-1], 'daily_return'])
        idx.append(times[-1])
        codes.append(code)
    return x_1d, y, idx, codes

In [9]:
for i in range(1, 4):
    # Generate training set samples
    x_train, y_train, idx_train, code_train = make_samples(globals()[f'train_data_{i}'], ret_1d, 5)
    np.save(f'data_rolling_3/x_train_{i}.npy', x_train)
    np.save(f'data_rolling_3/y_train_{i}.npy', y_train)
    np.save(f'data_rolling_3/idx_train_{i}.npy', idx_train)
    np.save(f'data_rolling_3/code_train_{i}.npy', code_train)

    # Generate validation set samples
    x_val, y_val, idx_val, code_val = make_samples(globals()[f'val_data_{i}'], ret_1d, 5)
    np.save(f'data_rolling_3/x_val_{i}.npy', x_val)
    np.save(f'data_rolling_3/y_val_{i}.npy', y_val)
    np.save(f'data_rolling_3/idx_val_{i}.npy', idx_val)
    np.save(f'data_rolling_3/code_val_{i}.npy', code_val)

    # Generate test set samples
    x_test, y_test, idx_test, code_test = make_samples(globals()[f'test_data_{i}'], ret_1d, 5)
    np.save(f'data_rolling_3/x_test_{i}.npy', x_test)
    np.save(f'data_rolling_3/y_test_{i}.npy', y_test)
    np.save(f'data_rolling_3/idx_test_{i}.npy', idx_test)
    np.save(f'data_rolling_3/code_test_{i}.npy', code_test)

    

make_sample: 100%|█████████████████████████| 8578/8578 [00:41<00:00, 205.01it/s]
